# Database Questions

*This worksheet was originally designed by [Erin George](https://www.math.ucla.edu/~egeo/) (Department of Mathematics, UCLA). It has been subsequently revised by later TAs and instructors.*

Today, we will be working with SQLite3 and Pandas to gain some practice working with large datasets.

Of course, the first thing we need to do is import our modules:

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt # optional, in case we want to plot

The dataset we will be working with is the US Wildfires dataset available on [kaggle](https://www.kaggle.com/rtatman/188-million-us-wildfires).  Downloading it from there requires a kaggle account.  You can either create a kaggle account or you can go on CCLE, where I've uploaded it.  Either way, put the database (.sqlite file) in the same directory as this worksheet, then run the following code block:

In [ ]:
conn = sqlite3.connect('FPA_FOD_20170508.sqlite')

A brief summary of the dataset: the data contains information on US wildfires between the years 1992 to 2015.  Each fire has information associated with it, such as how big it is, where it started, when it started, and so on.

This database contains A LOT of information:

In [ ]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

Fortunately, almost all the information we need is in the 'Fires' table!

Unfortunately, that means we wouldn't get any practice using joins.  Let's modify our data so that we split some information between separate tables in a reasonable way.

The code below is new, but it is just the command to create new tables from SELECT commands and the command to drop a table.  Once you run the code below, comment it out.  This will modify the database file and generate an error if the modification has already been made.

In [ ]:
q = cursor.execute("CREATE TABLE fire_sizes AS SELECT FOD_ID, FIRE_SIZE, FIRE_SIZE_CLASS FROM Fires;")
q = cursor.execute("CREATE TABLE fire_times AS SELECT FOD_ID, FIRE_YEAR, DISCOVERY_DATE, DISCOVERY_DOY, DISCOVERY_TIME, CONT_DATE, CONT_DOY, CONT_TIME FROM Fires;")
q = cursor.execute("CREATE TABLE fire_locs AS SELECT FOD_ID, LONGITUDE, LATITUDE, COUNTY, STATE FROM Fires;")
q = cursor.execute("CREATE TABLE fire_causes AS SELECT FOD_ID, STAT_CAUSE_CODE, STAT_CAUSE_DESCR FROM Fires;")
q = cursor.execute("DROP TABLE Fires;")

The tables now are `fire_sizes`, `fire_times`, `fire_locs` and `fire_causes`.  These tables contain the following information, organized by `FOD_ID`, which uniquely identifies each fire:
- `fire_size`: The size of the fire, in acres.  Also, the fire size class, which is a classification used to group up fires by size.
- `fire_times`: The date and time the fire was discovered and then controlled.  Also, the year the fire took place.
- `fire_locs`: The state, county, longitude, and latitude of the fire (where it started).
- `fire_causes`: The cause of each fire, as a code and a description.

Let's get some intial information from this table.  For example, we could ask the question "how many fires in this dataset were from California?"

Alternatively, we could ask a question such as "how many fires in this dataset happened since the year 2000?"

Now that we've split our data into multiple tables, some questions require information in two tables.  For example, "how many fires in this dataset happened in California since 2000?"

We can actually do the above in SQL itself with a clever choice of an accumulation function (COUNT):

## Group Questions

Let's ask a more complicated question.  Below is a list of three questions we can ask with this dataset.  You can decide which of the three questions you'd like to answer. In discussion, you'll then form groups of 2-3 people to work together on that question. These questions are intentionally open-ended, so write whatever code or do whatever data analysis you feel is necessary to answer the question.  Each question will have a few different subquestions to get you thinking of ideas: you can do any or all of them.  If your group finishes your question, go ahead and tackle another one!

Lastly, try to do the questions with as little non-SQL code as possible!  It's okay if you need more than just SQL, but you should be able to do at least one JOIN, GROUP BY, and accumulation in SQL before you do anything with Pandas to answer the subquestions.

**Question 1:** How has the intensity of the fires changed over time? (rows 1,2)

*Sub-question 1a:* What is the average size of a fire each year?

*Sub-question 1b:* How has the number of class G fires (the largest) changed over the years?

*Sub-question 1c:* What is the proportion of fires that are class A (the smallest) each year?

**Question 2:** How has the location of the fires changed over time? (rows 3,4)

*Sub-question 2a:* How has the average position of the fires changed over time?

*Sub-question 2b:* For each state, in what years did it have at least 4000 wildfires?

*Sub-question 2c:* What proportion of fires were in California each year?

**Question 3:** How do the causes of the fires vary by location? (rows 5,6)

*Sub-question 3a:* What's the most common cause of fire in each state? (This is much easier if you are familiar with `df.pivot_table` or `df.pivot`, although it is doable without both of them.)

*Sub-question 3b:* Which states have more fires from lightning than arson?

*Sub-question 3c:* What is the proportion of fires in each state that is caused by children? (These are labeled 'Children' in `STAT_CAUSE_DESCR`)